In [1]:
!pip install -q transformers langchain bitsandbytes

In [2]:
!pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.4.1.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.2 MB/s eta 0:00:00
Using cached scikit_learn-1.4.1.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)


In [3]:
!pip install llama_index

Defaulting to user installation because normal site-packages is not writeable


In [25]:
!pip install llama-index-embeddings-langchain

Defaulting to user installation because normal site-packages is not writeable


In [31]:
!pip install accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 2.0 MB/s eta 0:00:00a 0:00:01


In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.ollama import Ollama
from llama_index.core.prompts.prompts import SimpleInputPrompt


In [66]:
docs = SimpleDirectoryReader('/home/pacotoh/Documents/Data/data').load_data()
len(docs)

3

In [84]:
type(docs)

list

In [82]:
docs[0].metadata

{'file_path': '/home/pacotoh/Documents/Data/data/pg103.txt',
 'file_name': 'pg103.txt',
 'file_type': 'text/plain',
 'file_size': 403730,
 'creation_date': '2024-04-07',
 'last_modified_date': '2024-04-04'}

In [56]:
system_prompt = """
You are a Book information expert. Your goal is to answer questions as accurately as
possible based on the instructions and context provided. Do short answers to the questions
"""

query_wrapper_prompt = SimpleInputPrompt('<|USER|>{query_str}<|ASSISTANT|>')

In [50]:
llm = Ollama(model='llama2:13b', 
             query_wrapper_prompt=query_wrapper_prompt, 
             system_prompt=system_prompt)

In [88]:
llm

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7d12ace08490>, system_prompt='\nYou are a Book information expert. Your goal is to answer questions as accurately as\npossible based on the instructions and context provided. Do short answers to the questions\n', messages_to_prompt=<function messages_to_prompt at 0x7d132336a8e0>, completion_to_prompt=<function default_completion_to_prompt at 0x7d13233da2a0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='<|USER|>{query_str}<|ASSISTANT|>'), base_url='http://localhost:11434', model='llama2:13b', temperature=0.75, context_window=3900, request_timeout=30.0, prompt_key='prompt', additional_kwargs={})

In [51]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding (HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [52]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_433/186755162.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [53]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7d11c3ae4e50>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7d11c3ae4e50>, id_func=<function default_id_func at 0x7d132090f4c0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7d12aed3af90>, callback_manager=<llama_index.core.callbacks.ba

In [68]:
index = VectorStoreIndex.from_documents(docs, service_context=service_context)

In [76]:
query_engine = index.as_query_engine()
response = query_engine.query('Who is Jesuschrist?')

In [86]:
type(query_engine)

llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine

In [77]:
response

Response(response='Jesus Christ is not mentioned in the provided text context. The text discusses the lives of Eskimos and their conversion to Christianity by a missionary named Samuel Milliken Stewart. There is no mention of Jesus Christ or any other religious figures.', source_nodes=[NodeWithScore(node=TextNode(id_='4a845e9b-cc51-462a-93e2-1f5b7f7c4ba9', embedding=None, metadata={'file_path': '/home/pacotoh/Documents/Data/data/pg9857.txt', 'file_name': 'pg9857.txt', 'file_type': 'text/plain', 'file_size': 464336, 'creation_date': '2024-04-07', 'last_modified_date': '2024-04-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3ddce1d4-ea47-4a1c-b115-86da35fb77ee', node_type=<ObjectType.DOCUMENT: '4'>, me

In [87]:
type(index)

llama_index.core.indices.vector_store.base.VectorStoreIndex